In [2]:
# !git clone https://github.com/bennokr/rdflib-r2r.git
# !pip install ./rdflib-r2r

In [3]:
from rdflib_r2r import R2RStore, R2RMapping
from sqlalchemy import create_engine
from pathlib import Path
import rdflib
from rdflib.compare import to_isomorphic, graph_diff

echo = False


def create_database(db, path, *sql_fnames):
    for sql_fname in sql_fnames:
        sql_script = (path / sql_fname).open().read()
        conn = db.raw_connection()
        try:
            if hasattr(conn.connection, "c"):
                cursor = conn.connection.c
            else:
                cursor = conn.cursor()
            cursor.execute(sql_script, (), None)
        except Exception as e:
            try:
                cursor = conn.cursor()
                cursor.executescript(sql_script)
            except Exception as e:
                raise Exception(f"Problem with {path}: {e}")
            finally:
                conn.close()
        finally:
            conn.close()

# db = create_engine(
#     "duckdb:///:memory:",
#     echo=echo,
#     # future=True,
#     connect_args={"read_only": False},
# )
# db.dialect.server_version_info = (0,)

import sqlite3
db = create_engine(
    "sqlite:///:memory:",
    echo=echo,
    future=True,
    connect_args={
        "detect_types": sqlite3.PARSE_DECLTYPES | sqlite3.PARSE_COLNAMES,
        'timeout': 30 * 1000
    },
    native_datetime=False, # breaks rdb2rdf test D016
    # native_datetime=True, # breaks BSBM test 3
)

# path = Path('rdflib-r2r/tests/rdb2rdf-ts/D014-3tables1primarykey1foreignkey')
path = Path('tests/rdb2rdf-ts/D014-3tables1primarykey1foreignkey')
sql_fname = 'create.sql'

sql_script = (path / sql_fname).open().read()
create_database(db, path, sql_fname)

# Loads R2RML mapping
mapfile = path / 'r2rmlb.ttl'
fmt = rdflib.util.guess_format(str(mapfile))
mapping = R2RMapping(rdflib.Graph().parse(str(mapfile), format=fmt))

# Creates Direct Mapping from DB schema
# mapping = R2RMapping.from_db(db)

g_made = rdflib.Graph(R2RStore(db=db, mapping=mapping))
print(to_isomorphic(g_made).serialize())

query = g_made.store.getSQL("SELECT * WHERE {?s ?p ?o}")
# db.raw_connection().cursor().execute("SELECT * FROM DEPT").fetchall()
db.raw_connection().cursor().execute(query).fetchall()

/var/folders/jd/y6jxdm1s23l3qtl8w9js19880000gn/T/ipykernel_39079/2757565334.py:15: SADeprecationWarning: The _ConnectionFairy.connection attribute is deprecated; please use 'driver_connection' (deprecated since: 2.0)
  if hasattr(conn.connection, "c"):
/Users/vickyli117/Desktop/rdflib-r2r/rdflib_r2r/r2r_store.py:672: SAWarning: SELECT statement has a cartesian product between FROM element(s) "View_NB2HI4B2F4XWK6DBNVYGYZJOMNXW2L3CMFZWKL2UOJUXA3DFONGWC4BR_ref" and FROM element "View_NB2HI4B2F4XWK6DBNVYGYZJOMNXW2L3CMFZWKL2UOJUXA3DFONGWC4BS".  Apply join condition(s) between each element to resolve.
  rows = list(conn.execute(query))


@prefix ns1: <http://example.com/emp#> .
@prefix ns2: <http://example.com/dept#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<http://example.com/emp/7369> a <http://example.com/emp/etype/PART_TIME>,
        <http://example.com/emp/job/CLERK> ;
    ns1:c_ref_deptno [ ns2:COMPANY "EXAMPLE Corporation" ;
            ns2:deptno 10 ;
            ns2:location "NEW YORK" ;
            ns2:name "APPSERVER" ] ;
    ns1:deptNum 10 ;
    ns1:empno 7369 ;
    ns1:emptype <http://example.com/emp/etype/PART_TIME> ;
    ns1:etype "PART_TIME" ;
    ns1:job "CLERK" ;
    ns1:jobtype <http://example.com/emp/job/CLERK> ;
    ns1:name "SMITH" .




[('_:Department10',
  '<http://example.com/dept#COMPANY>',
  '"EXAMPLE Corporation"'),
 ('_:Department10', '<http://example.com/dept#deptno>', 10),
 ('_:Department10', '<http://example.com/dept#location>', 'NEW YORK'),
 ('_:Department10', '<http://example.com/dept#name>', 'APPSERVER'),
 ('<http://example.com/emp/7369>', '<http://example.com/emp#job>', 'CLERK'),
 ('<http://example.com/emp/7369>',
  '<http://example.com/emp#etype>',
  'PART_TIME'),
 ('<http://example.com/emp/7369>', '<http://example.com/emp#name>', 'SMITH'),
 ('<http://example.com/emp/7369>',
  '<http://example.com/emp#c_ref_deptno>',
  '_:Department10'),
 ('<http://example.com/emp/7369>',
  '<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>',
  '<http://example.com/emp/job/CLERK>'),
 ('<http://example.com/emp/7369>',
  '<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>',
  '<http://example.com/emp/etype/PART_TIME>'),
 ('<http://example.com/emp/7369>',
  '<http://example.com/emp#jobtype>',
  '<http://example.com/emp/job/

In [4]:
query = g_made.store.getSQL("SELECT * WHERE {?s ?p \"NEW YORK\"}")
print(query)
# db.raw_connection().cursor().execute("SELECT * FROM DEPT").fetchall()
db.raw_connection().cursor().execute(query).fetchall()

SELECT anon_1.s AS s,
       anon_1.p AS p
FROM
  (SELECT '_:' || CAST("View_NB2HI4B2F4XWK6DBNVYGYZJOMNXW2L3CMFZWKL2UOJUXA3DFONGWC4BR".deptId AS VARCHAR) AS s,
          '<http://example.com/dept#deptno>' AS p,
          "View_NB2HI4B2F4XWK6DBNVYGYZJOMNXW2L3CMFZWKL2UOJUXA3DFONGWC4BR"."deptno" AS o,
          NULL AS g
   FROM
     (SELECT ('Department' || "deptno") AS deptId ,
             "deptno" ,
             "dname" ,
             "loc"
      FROM "DEPT") AS "View_NB2HI4B2F4XWK6DBNVYGYZJOMNXW2L3CMFZWKL2UOJUXA3DFONGWC4BR"
   WHERE "View_NB2HI4B2F4XWK6DBNVYGYZJOMNXW2L3CMFZWKL2UOJUXA3DFONGWC4BR"."deptno" = 'NEW YORK'
   UNION ALL SELECT '_:' || CAST("View_NB2HI4B2F4XWK6DBNVYGYZJOMNXW2L3CMFZWKL2UOJUXA3DFONGWC4BR".deptId AS VARCHAR) AS s,
                    '<http://example.com/dept#location>' AS p,
                    "View_NB2HI4B2F4XWK6DBNVYGYZJOMNXW2L3CMFZWKL2UOJUXA3DFONGWC4BR"."loc" AS o,
                    NULL AS g
   FROM
     (SELECT ('Department' || "deptno") AS deptId ,
  

OperationalError: ambiguous column name: deptno

In [7]:
newquery = """
SELECT '<' || CAST("View_NB2HI4B2F4XWK6DBNVYGYZJOMNXW2L3CMFZWKL2UOJUXA3DFONGWC4BS".empURI AS VARCHAR) || '>' AS s,
                    '<http://example.com/emp#c_ref_deptno>' AS p,
                    '_:' || CAST("View_NB2HI4B2F4XWK6DBNVYGYZJOMNXW2L3CMFZWKL2UOJUXA3DFONGWC4BR_ref".deptId AS VARCHAR) AS o,
                    NULL AS g
   FROM
     (SELECT ('http://example.com/emp/' || "empno") AS empURI ,
             "empno" ,
             "ename" ,
             ('http://example.com/emp/job/'|| "job") AS jobTypeURI ,
             "job" ,
             "deptno" ,
             ('http://example.com/emp/etype/'|| "etype") AS empTypeURI ,
             "etype" ,
             ('http://example.com/graph/'|| "job" || '/' || "etype") AS graphURI
      FROM "EMP") AS "View_NB2HI4B2F4XWK6DBNVYGYZJOMNXW2L3CMFZWKL2UOJUXA3DFONGWC4BS",

     (SELECT ('Department' || "deptno") AS deptId ,
             "deptno" ,
             "dname" ,
             "loc"
      FROM "DEPT") AS "View_NB2HI4B2F4XWK6DBNVYGYZJOMNXW2L3CMFZWKL2UOJUXA3DFONGWC4BR_ref"
   WHERE "deptno" = substr('NEW YORK', length('Department')+1)
     AND "View_NB2HI4B2F4XWK6DBNVYGYZJOMNXW2L3CMFZWKL2UOJUXA3DFONGWC4BS"."deptno" = "View_NB2HI4B2F4XWK6DBNVYGYZJOMNXW2L3CMFZWKL2UOJUXA3DFONGWC4BR_ref"."deptno"
"""
db.raw_connection().cursor().execute(newquery).fetchall()   

[]

In [40]:
newquery = """
SELECT '_:' || CAST("View_NB2HI4B2F4XWK6DBNVYGYZJOMNXW2L3CMFZWKL2UOJUXA3DFONGWC4BR".deptId AS VARCHAR) AS s,
          '<http://example.com/dept#location>' AS p,
          "View_NB2HI4B2F4XWK6DBNVYGYZJOMNXW2L3CMFZWKL2UOJUXA3DFONGWC4BR"."loc" AS o,
          NULL AS g
   FROM
     (SELECT ('Department' || "deptno") AS deptId ,
             "deptno" ,
             "dname" ,
             "loc"
      FROM "DEPT") AS "View_NB2HI4B2F4XWK6DBNVYGYZJOMNXW2L3CMFZWKL2UOJUXA3DFONGWC4BR"
   WHERE "View_NB2HI4B2F4XWK6DBNVYGYZJOMNXW2L3CMFZWKL2UOJUXA3DFONGWC4BR"."loc" = 'NEW YORK'
"""
db.raw_connection().cursor().execute(newquery).fetchall() 

[('_:Department10', '<http://example.com/dept#location>', 'NEW YORK', None)]

In [41]:
newquery = """
SELECT anon_1.s AS s,
       anon_1.p AS p
FROM
  (SELECT '_:' || CAST("View_NB2HI4B2F4XWK6DBNVYGYZJOMNXW2L3CMFZWKL2UOJUXA3DFONGWC4BR".deptId AS VARCHAR) AS s,
          '<http://example.com/dept#location>' AS p,
          "View_NB2HI4B2F4XWK6DBNVYGYZJOMNXW2L3CMFZWKL2UOJUXA3DFONGWC4BR"."loc" AS o,
          NULL AS g
   FROM
     (SELECT ('Department' || "deptno") AS deptId ,
             "deptno" ,
             "dname" ,
             "loc"
      FROM "DEPT") AS "View_NB2HI4B2F4XWK6DBNVYGYZJOMNXW2L3CMFZWKL2UOJUXA3DFONGWC4BR"
   WHERE "View_NB2HI4B2F4XWK6DBNVYGYZJOMNXW2L3CMFZWKL2UOJUXA3DFONGWC4BR"."loc" = 'NEW YORK'
   UNION ALL SELECT '_:' || CAST("View_NB2HI4B2F4XWK6DBNVYGYZJOMNXW2L3CMFZWKL2UOJUXA3DFONGWC4BR".deptId AS VARCHAR) AS s,
                    '<http://example.com/dept#name>' AS p,
                    "View_NB2HI4B2F4XWK6DBNVYGYZJOMNXW2L3CMFZWKL2UOJUXA3DFONGWC4BR"."dname" AS o,
                    NULL AS g
   FROM
     (SELECT ('Department' || "deptno") AS deptId ,
             "deptno" ,
             "dname" ,
             "loc"
      FROM "DEPT") AS "View_NB2HI4B2F4XWK6DBNVYGYZJOMNXW2L3CMFZWKL2UOJUXA3DFONGWC4BR"
   WHERE "View_NB2HI4B2F4XWK6DBNVYGYZJOMNXW2L3CMFZWKL2UOJUXA3DFONGWC4BR"."dname" = 'NEW YORK'
   UNION ALL SELECT '_:' || CAST("View_NB2HI4B2F4XWK6DBNVYGYZJOMNXW2L3CMFZWKL2UOJUXA3DFONGWC4BR".deptId AS VARCHAR) AS s,
                    '<http://example.com/dept#deptno>' AS p,
                    "View_NB2HI4B2F4XWK6DBNVYGYZJOMNXW2L3CMFZWKL2UOJUXA3DFONGWC4BR"."deptno" AS o,
                    NULL AS g
   FROM
     (SELECT ('Department' || "deptno") AS deptId ,
             "deptno" ,
             "dname" ,
             "loc"
      FROM "DEPT") AS "View_NB2HI4B2F4XWK6DBNVYGYZJOMNXW2L3CMFZWKL2UOJUXA3DFONGWC4BR"
   WHERE "View_NB2HI4B2F4XWK6DBNVYGYZJOMNXW2L3CMFZWKL2UOJUXA3DFONGWC4BR"."deptno" = 'NEW YORK'
   UNION ALL SELECT '<' || CAST("View_NB2HI4B2F4XWK6DBNVYGYZJOMNXW2L3CMFZWKL2UOJUXA3DFONGWC4BS".empURI AS VARCHAR) || '>' AS s,
                    '<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>' AS p,
                    '<' || CAST("View_NB2HI4B2F4XWK6DBNVYGYZJOMNXW2L3CMFZWKL2UOJUXA3DFONGWC4BS".empTypeURI AS VARCHAR) || '>' AS o,
                    NULL AS g
   FROM
     (SELECT ('http://example.com/emp/' || "empno") AS empURI ,
             "empno" ,
             "ename" ,
             ('http://example.com/emp/job/'|| "job") AS jobTypeURI ,
             "job" ,
             "deptno" ,
             ('http://example.com/emp/etype/'|| "etype") AS empTypeURI ,
             "etype" ,
             ('http://example.com/graph/'|| "job" || '/' || "etype") AS graphURI
      FROM "EMP") AS "View_NB2HI4B2F4XWK6DBNVYGYZJOMNXW2L3CMFZWKL2UOJUXA3DFONGWC4BS"
   WHERE "View_NB2HI4B2F4XWK6DBNVYGYZJOMNXW2L3CMFZWKL2UOJUXA3DFONGWC4BS"."empTypeURI" = 'NEW YORK'
   UNION ALL SELECT '<' || CAST("View_NB2HI4B2F4XWK6DBNVYGYZJOMNXW2L3CMFZWKL2UOJUXA3DFONGWC4BS".empURI AS VARCHAR) || '>' AS s,
                    '<http://example.com/emp#deptNum>' AS p,
                    "View_NB2HI4B2F4XWK6DBNVYGYZJOMNXW2L3CMFZWKL2UOJUXA3DFONGWC4BS"."deptno" AS o,
                    NULL AS g
   FROM
     (SELECT ('http://example.com/emp/' || "empno") AS empURI ,
             "empno" ,
             "ename" ,
             ('http://example.com/emp/job/'|| "job") AS jobTypeURI ,
             "job" ,
             "deptno" ,
             ('http://example.com/emp/etype/'|| "etype") AS empTypeURI ,
             "etype" ,
             ('http://example.com/graph/'|| "job" || '/' || "etype") AS graphURI
      FROM "EMP") AS "View_NB2HI4B2F4XWK6DBNVYGYZJOMNXW2L3CMFZWKL2UOJUXA3DFONGWC4BS"
   WHERE "View_NB2HI4B2F4XWK6DBNVYGYZJOMNXW2L3CMFZWKL2UOJUXA3DFONGWC4BS"."deptno" = 'NEW YORK'
   UNION ALL SELECT '<' || CAST("View_NB2HI4B2F4XWK6DBNVYGYZJOMNXW2L3CMFZWKL2UOJUXA3DFONGWC4BS".empURI AS VARCHAR) || '>' AS s,
                    '<http://example.com/emp#etype>' AS p,
                    "View_NB2HI4B2F4XWK6DBNVYGYZJOMNXW2L3CMFZWKL2UOJUXA3DFONGWC4BS"."etype" AS o,
                    NULL AS g
   FROM
     (SELECT ('http://example.com/emp/' || "empno") AS empURI ,
             "empno" ,
             "ename" ,
             ('http://example.com/emp/job/'|| "job") AS jobTypeURI ,
             "job" ,
             "deptno" ,
             ('http://example.com/emp/etype/'|| "etype") AS empTypeURI ,
             "etype" ,
             ('http://example.com/graph/'|| "job" || '/' || "etype") AS graphURI
      FROM "EMP") AS "View_NB2HI4B2F4XWK6DBNVYGYZJOMNXW2L3CMFZWKL2UOJUXA3DFONGWC4BS"
   WHERE "View_NB2HI4B2F4XWK6DBNVYGYZJOMNXW2L3CMFZWKL2UOJUXA3DFONGWC4BS"."etype" = 'NEW YORK'
   UNION ALL SELECT '<' || CAST("View_NB2HI4B2F4XWK6DBNVYGYZJOMNXW2L3CMFZWKL2UOJUXA3DFONGWC4BS".empURI AS VARCHAR) || '>' AS s,
                    '<http://example.com/emp#emptype>' AS p,
                    '<' || CAST("View_NB2HI4B2F4XWK6DBNVYGYZJOMNXW2L3CMFZWKL2UOJUXA3DFONGWC4BS".empTypeURI AS VARCHAR) || '>' AS o,
                    NULL AS g
   FROM
     (SELECT ('http://example.com/emp/' || "empno") AS empURI ,
             "empno" ,
             "ename" ,
             ('http://example.com/emp/job/'|| "job") AS jobTypeURI ,
             "job" ,
             "deptno" ,
             ('http://example.com/emp/etype/'|| "etype") AS empTypeURI ,
             "etype" ,
             ('http://example.com/graph/'|| "job" || '/' || "etype") AS graphURI
      FROM "EMP") AS "View_NB2HI4B2F4XWK6DBNVYGYZJOMNXW2L3CMFZWKL2UOJUXA3DFONGWC4BS"
   WHERE "View_NB2HI4B2F4XWK6DBNVYGYZJOMNXW2L3CMFZWKL2UOJUXA3DFONGWC4BS"."empTypeURI" = 'NEW YORK'
   UNION ALL SELECT '<' || CAST("View_NB2HI4B2F4XWK6DBNVYGYZJOMNXW2L3CMFZWKL2UOJUXA3DFONGWC4BS".empURI AS VARCHAR) || '>' AS s,
                    '<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>' AS p,
                    '<' || CAST("View_NB2HI4B2F4XWK6DBNVYGYZJOMNXW2L3CMFZWKL2UOJUXA3DFONGWC4BS".jobTypeURI AS VARCHAR) || '>' AS o,
                    NULL AS g
   FROM
     (SELECT ('http://example.com/emp/' || "empno") AS empURI ,
             "empno" ,
             "ename" ,
             ('http://example.com/emp/job/'|| "job") AS jobTypeURI ,
             "job" ,
             "deptno" ,
             ('http://example.com/emp/etype/'|| "etype") AS empTypeURI ,
             "etype" ,
             ('http://example.com/graph/'|| "job" || '/' || "etype") AS graphURI
      FROM "EMP") AS "View_NB2HI4B2F4XWK6DBNVYGYZJOMNXW2L3CMFZWKL2UOJUXA3DFONGWC4BS"
   WHERE "View_NB2HI4B2F4XWK6DBNVYGYZJOMNXW2L3CMFZWKL2UOJUXA3DFONGWC4BS"."jobTypeURI" = 'NEW YORK'
   UNION ALL SELECT '<' || CAST("View_NB2HI4B2F4XWK6DBNVYGYZJOMNXW2L3CMFZWKL2UOJUXA3DFONGWC4BS".empURI AS VARCHAR) || '>' AS s,
                    '<http://example.com/emp#c_ref_deptno>' AS p,
                    '_:' || CAST("View_NB2HI4B2F4XWK6DBNVYGYZJOMNXW2L3CMFZWKL2UOJUXA3DFONGWC4BR_ref".deptId AS VARCHAR) AS o,
                    NULL AS g
   FROM
     (SELECT ('http://example.com/emp/' || "empno") AS empURI ,
             "empno" ,
             "ename" ,
             ('http://example.com/emp/job/'|| "job") AS jobTypeURI ,
             "job" ,
             "deptno" ,
             ('http://example.com/emp/etype/'|| "etype") AS empTypeURI ,
             "etype" ,
             ('http://example.com/graph/'|| "job" || '/' || "etype") AS graphURI
      FROM "EMP") AS "View_NB2HI4B2F4XWK6DBNVYGYZJOMNXW2L3CMFZWKL2UOJUXA3DFONGWC4BS",

     (SELECT ('Department' || "deptno") AS deptId ,
             "deptno" ,
             "dname" ,
             "loc"
      FROM "DEPT") AS "View_NB2HI4B2F4XWK6DBNVYGYZJOMNXW2L3CMFZWKL2UOJUXA3DFONGWC4BR_ref"
   WHERE "loc" = substr('NEW YORK', length('Department')+1)
     AND "View_NB2HI4B2F4XWK6DBNVYGYZJOMNXW2L3CMFZWKL2UOJUXA3DFONGWC4BS"."deptno" = "View_NB2HI4B2F4XWK6DBNVYGYZJOMNXW2L3CMFZWKL2UOJUXA3DFONGWC4BR_ref"."deptno"
   UNION ALL SELECT '<' || CAST("View_NB2HI4B2F4XWK6DBNVYGYZJOMNXW2L3CMFZWKL2UOJUXA3DFONGWC4BS".empURI AS VARCHAR) || '>' AS s,
                    '<http://example.com/emp#jobtype>' AS p,
                    '<' || CAST("View_NB2HI4B2F4XWK6DBNVYGYZJOMNXW2L3CMFZWKL2UOJUXA3DFONGWC4BS".jobTypeURI AS VARCHAR) || '>' AS o,
                    NULL AS g
   FROM
     (SELECT ('http://example.com/emp/' || "empno") AS empURI ,
             "empno" ,
             "ename" ,
             ('http://example.com/emp/job/'|| "job") AS jobTypeURI ,
             "job" ,
             "deptno" ,
             ('http://example.com/emp/etype/'|| "etype") AS empTypeURI ,
             "etype" ,
             ('http://example.com/graph/'|| "job" || '/' || "etype") AS graphURI
      FROM "EMP") AS "View_NB2HI4B2F4XWK6DBNVYGYZJOMNXW2L3CMFZWKL2UOJUXA3DFONGWC4BS"
   WHERE "View_NB2HI4B2F4XWK6DBNVYGYZJOMNXW2L3CMFZWKL2UOJUXA3DFONGWC4BS"."jobTypeURI" = 'NEW YORK'
   UNION ALL SELECT '<' || CAST("View_NB2HI4B2F4XWK6DBNVYGYZJOMNXW2L3CMFZWKL2UOJUXA3DFONGWC4BS".empURI AS VARCHAR) || '>' AS s,
                    '<http://example.com/emp#empno>' AS p,
                    "View_NB2HI4B2F4XWK6DBNVYGYZJOMNXW2L3CMFZWKL2UOJUXA3DFONGWC4BS"."empno" AS o,
                    NULL AS g
   FROM
     (SELECT ('http://example.com/emp/' || "empno") AS empURI ,
             "empno" ,
             "ename" ,
             ('http://example.com/emp/job/'|| "job") AS jobTypeURI ,
             "job" ,
             "deptno" ,
             ('http://example.com/emp/etype/'|| "etype") AS empTypeURI ,
             "etype" ,
             ('http://example.com/graph/'|| "job" || '/' || "etype") AS graphURI
      FROM "EMP") AS "View_NB2HI4B2F4XWK6DBNVYGYZJOMNXW2L3CMFZWKL2UOJUXA3DFONGWC4BS"
   WHERE "View_NB2HI4B2F4XWK6DBNVYGYZJOMNXW2L3CMFZWKL2UOJUXA3DFONGWC4BS"."empno" = 'NEW YORK'
   UNION ALL SELECT '<' || CAST("View_NB2HI4B2F4XWK6DBNVYGYZJOMNXW2L3CMFZWKL2UOJUXA3DFONGWC4BS".empURI AS VARCHAR) || '>' AS s,
                    '<http://example.com/emp#job>' AS p,
                    "View_NB2HI4B2F4XWK6DBNVYGYZJOMNXW2L3CMFZWKL2UOJUXA3DFONGWC4BS"."job" AS o,
                    NULL AS g
   FROM
     (SELECT ('http://example.com/emp/' || "empno") AS empURI ,
             "empno" ,
             "ename" ,
             ('http://example.com/emp/job/'|| "job") AS jobTypeURI ,
             "job" ,
             "deptno" ,
             ('http://example.com/emp/etype/'|| "etype") AS empTypeURI ,
             "etype" ,
             ('http://example.com/graph/'|| "job" || '/' || "etype") AS graphURI
      FROM "EMP") AS "View_NB2HI4B2F4XWK6DBNVYGYZJOMNXW2L3CMFZWKL2UOJUXA3DFONGWC4BS"
   WHERE "View_NB2HI4B2F4XWK6DBNVYGYZJOMNXW2L3CMFZWKL2UOJUXA3DFONGWC4BS"."job" = 'NEW YORK'
   UNION ALL SELECT '<' || CAST("View_NB2HI4B2F4XWK6DBNVYGYZJOMNXW2L3CMFZWKL2UOJUXA3DFONGWC4BS".empURI AS VARCHAR) || '>' AS s,
                    '<http://example.com/emp#name>' AS p,
                    "View_NB2HI4B2F4XWK6DBNVYGYZJOMNXW2L3CMFZWKL2UOJUXA3DFONGWC4BS"."ename" AS o,
                    NULL AS g
   FROM
     (SELECT ('http://example.com/emp/' || "empno") AS empURI ,
             "empno" ,
             "ename" ,
             ('http://example.com/emp/job/'|| "job") AS jobTypeURI ,
             "job" ,
             "deptno" ,
             ('http://example.com/emp/etype/'|| "etype") AS empTypeURI ,
             "etype" ,
             ('http://example.com/graph/'|| "job" || '/' || "etype") AS graphURI
      FROM "EMP") AS "View_NB2HI4B2F4XWK6DBNVYGYZJOMNXW2L3CMFZWKL2UOJUXA3DFONGWC4BS"
   WHERE "View_NB2HI4B2F4XWK6DBNVYGYZJOMNXW2L3CMFZWKL2UOJUXA3DFONGWC4BS"."ename" = 'NEW YORK') AS anon_1
"""
db.raw_connection().cursor().execute(newquery).fetchall()

[('_:Department10', '<http://example.com/dept#location>')]

In [8]:
o = rdflib.term.Literal('7369', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#integer'))
#o = rdflib.term.Literal('NEW YORK')
list(g_made.triples([None, None, o]))

OperationalError: (sqlite3.OperationalError) ambiguous column name: deptno
[SQL: SELECT anon_1.s AS s, anon_1.p AS p, anon_1.o AS o, anon_1.g AS g 
FROM (SELECT ? || CAST("View_NB2HI4B2F4XWK6DBNVYGYZJOMNXW2L3CMFZWKL2UOJUXA3DFONGWC4BR".deptId AS VARCHAR) AS s, '<http://example.com/dept#deptno>' AS p, "View_NB2HI4B2F4XWK6DBNVYGYZJOMNXW2L3CMFZWKL2UOJUXA3DFONGWC4BR"."deptno" AS o, NULL AS g 
FROM (SELECT ('Department' || "deptno") AS deptId
            , "deptno"
            , "dname"
            , "loc"
       FROM "DEPT") AS "View_NB2HI4B2F4XWK6DBNVYGYZJOMNXW2L3CMFZWKL2UOJUXA3DFONGWC4BR" 
WHERE "View_NB2HI4B2F4XWK6DBNVYGYZJOMNXW2L3CMFZWKL2UOJUXA3DFONGWC4BR"."deptno" = ? UNION ALL SELECT ? || CAST("View_NB2HI4B2F4XWK6DBNVYGYZJOMNXW2L3CMFZWKL2UOJUXA3DFONGWC4BR".deptId AS VARCHAR) AS s, '<http://example.com/dept#location>' AS p, "View_NB2HI4B2F4XWK6DBNVYGYZJOMNXW2L3CMFZWKL2UOJUXA3DFONGWC4BR"."loc" AS o, NULL AS g 
FROM (SELECT ('Department' || "deptno") AS deptId
            , "deptno"
    

In [7]:
print(g_made.store.getSQL("SELECT * WHERE {?s ?p 7369}"))

SELECT anon_1.s AS s,
       anon_1.p AS p
FROM
  (SELECT '_:' || CAST("View_NB2HI4B2F4XWK6DBNVYGYZJOMNXW2L3CMFZWKL2UOJUXA3DFONGWC4BR".deptId AS VARCHAR) AS s,
          '<http://example.com/dept#deptno>' AS p,
          "View_NB2HI4B2F4XWK6DBNVYGYZJOMNXW2L3CMFZWKL2UOJUXA3DFONGWC4BR"."deptno" AS o,
          NULL AS g
   FROM
     (SELECT ('Department' || "deptno") AS deptId ,
             "deptno" ,
             "dname" ,
             "loc"
      FROM "DEPT") AS "View_NB2HI4B2F4XWK6DBNVYGYZJOMNXW2L3CMFZWKL2UOJUXA3DFONGWC4BR"
   WHERE "View_NB2HI4B2F4XWK6DBNVYGYZJOMNXW2L3CMFZWKL2UOJUXA3DFONGWC4BR"."deptno" = 7369
   UNION ALL SELECT '_:' || CAST("View_NB2HI4B2F4XWK6DBNVYGYZJOMNXW2L3CMFZWKL2UOJUXA3DFONGWC4BR".deptId AS VARCHAR) AS s,
                    '<http://example.com/dept#location>' AS p,
                    "View_NB2HI4B2F4XWK6DBNVYGYZJOMNXW2L3CMFZWKL2UOJUXA3DFONGWC4BR"."loc" AS o,
                    NULL AS g
   FROM
     (SELECT ('Department' || "deptno") AS deptId ,
        